# Open Rocket Engine Development Notebook

This notebook will temporarily be used to calculate engine parameters. Equations, classes, etc. built here will eventually be moved to the enginebuilder module

In [1]:
import numpy as np

### Useful Functions

In [2]:
def atm_to_psi(value):
    return 14.6959*value

def psi_to_atm(value):
    return value/14.6959

def kelvin_to_rankine(value):
    return value*9/5

def rankine_to_kelvin(value):
    return value*5/9

### Design Requirements and Fuel Properties
User entered values.

Propellant values are determined using NASA CEA and Braeuing
http://www.braeunig.us/space/comb-OM.htm
https://www.grc.nasa.gov/www/CEAWeb/

In [3]:
thrust = 1000  # [lbf] thrust
Pchamber = 300 # [psi] chamber pressure
Pambient = 14.6959  # [psi] ambient pressure

# Propellant Properties
MR = 2.60  # [1] Mixture Ratio (O:F)
Tchamber = kelvin_to_rankine(3200) # [K]
gamma = 1.215  # specific heat ratio
MW_products = 18.8  # Molecular Weight of Combustion Products *VERY APPROXIMATE*

In [4]:
psi_to_atm(Pchamber)

20.413856926081426

### Some Constants

In [5]:
g0 = 32.174
Rspecific_fuel = 18544.2/MW_fuel
Rspecific_ox = 18544.2/MW_ox

NameError: name 'MW_fuel' is not defined

### Thrust-Chamber Performance Parameters